## Cleaning Credit Card Transaction Dataset 

In [1]:
# Libraries to install
# %pip install pandas-profiling

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from datetime import timedelta

Load in the data.

In [3]:
%%time
mydata = pd.read_csv('card transactions.csv')

CPU times: user 132 ms, sys: 30.9 ms, total: 163 ms
Wall time: 183 ms


Drop unwanted columns.

In [4]:
mydata.drop(['Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17'],axis=1, inplace=True)

Change Date to a datetime object

In [5]:
mydata['Date']= pd.to_datetime(mydata['Date'])

In [6]:
mydata.dtypes

Recnum                        int64
Cardnum                       int64
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
Merch state                  object
Merch zip                   float64
Transtype                    object
Amount                      float64
Fraud                         int64
dtype: object

### Removing single large outlier

Need to identify what record the outlier is. Know transaction amount is above one million.

In [7]:
mydata[mydata['Amount'] > 1000000]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
52714,52715,5142189135,2010-07-13,NaN,INTERMEXICO,NaN,NaN,P,3102045.53,0


Drop this row from the data set.

In [8]:
mydata.drop([52714], inplace=True)

Check to make sure it has been removed.

In [9]:
mydata[mydata['Amount'] > 1000000]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud


### Removing all but the "P" type transactions

See what the different transaction types are.

In [10]:
mydata["Transtype"].value_counts()

P    96397
A      181
D      173
Y        1
Name: Transtype, dtype: int64

Too many rows to drop, so let's create a new dataset with only 'P'.

In [11]:
mydata = mydata[mydata["Transtype"]=='P']

In [12]:
mydata.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


New data set. Removed outlier. Removed all transaction types but 'P'.

# Filling in Missing Records

### Merchnum

If merchnum == 0, replace with NaN

In [13]:
len(mydata[mydata['Merchnum']=='0'])

53

In [14]:
mydata.loc[mydata['Merchnum']== '0','Merchnum']=np.nan

In [15]:
len(mydata[mydata['Merchnum']=='0'])

0

Identify the location of the null values.

In [16]:
null_merchnum = mydata[mydata['Merchnum'].isnull()]
print(null_merchnum.index)
null_merchnum.head()

Int64Index([  115,   135,   168,   201,   257,   260,   262,   272,   328,
              358,
            ...
            96509, 96558, 96563, 96586, 96603, 96621, 96651, 96689, 96720,
            96730],
           dtype='int64', length=3251)


,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
115,116,5142182128,2010-01-03,NaN,GRAINGER #973,IL,60089.0,P,327.34,0
135,136,5142126842,2010-01-03,NaN,AUTOMATED OFFICE PRODU,MD,20706.0,P,2110.00,0
168,169,5142132574,2010-01-03,NaN,ROLL CALL NEWSPAPER,DC,20001.0,P,104.69,0
201,202,5142140436,2010-01-03,NaN,MONTGOMERY COLLEGE-PHONE,MD,20850.0,P,266.00,0
257,258,5142257292,2010-01-04,NaN,ARGENT CHEMICAL LABS,WA,98052.0,P,240.00,0


Let's see if any of the records missing merchnum share a merch description with other records. 

In [17]:
temp = mydata[mydata['Merchnum'].notnull()]
temp.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [18]:
temp = temp.groupby(['Merch description'])['Merchnum'].unique().to_frame()
temp

,Merchnum
Merch description,
#9 SOFTWARE,[6000330882278]
(ISC)2 CERTIFICATION,[590065510]
0000000000000000000000,[8168600400097]
033007 KINKO'S,[35068136338]
05032 FLYING J,[121075089FL96]
...,...
ZYMARK CORPORATION,"[790800099272, 988906278331]"
ZYMAX ENVIROTECHNOLOGY,[9612006916900]
ZYMED LABORATORIES,[679966068337]


If a record missing the merchnum has the same merch description as another transaction in the dataset, then we are going to set the merchnum equal to what is found in the other transaction.

In [19]:
for index in null_merchnum.index:
    if mydata.loc[index, 'Merch description'] in temp.index:
        merch_desc = mydata.loc[index, 'Merch description']
        mydata.loc[index, 'Merchnum'] = temp.loc[merch_desc, 'Merchnum'][0]

We will still have records missing merchnum.

In [20]:
null_merchnum = mydata[mydata['Merchnum'].isnull()]
print(len(null_merchnum))
null_merchnum.head()

2094


,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
201,202,5142140436,2010-01-03,NaN,MONTGOMERY COLLEGE-PHONE,MD,20850.0,P,266.00,0
262,263,5142257575,2010-01-04,NaN,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,320.00,0
272,273,5142124791,2010-01-04,NaN,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,970.00,0
400,401,5142276099,2010-01-04,NaN,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,82.59,0
476,477,5142267793,2010-01-05,NaN,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,17.59,0


Fill the rest of the missing values in with the mode of merch description.

In [21]:
# mode_merchdesc = mydata.groupby('Merch description')['Merchnum'].value_counts().sort_values(ascending=False)
# print(mode_merchdesc.head(3))

In [22]:
# for index in null_merchnum.index:
#         mydata.loc[index,'Merchnum'] = '9900020006406'

In [23]:
merchdesc_merchnum = {}
for index,merchdesc in mydata[mydata['Merch description'].notnull()]['Merch description'].items():
    if merchdesc not in merchdesc_merchnum:
        merchdesc_merchnum[merchdesc] = mydata.loc[index, 'Merchnum']

In [24]:
mydata['Merchnum'] = mydata['Merchnum'].fillna(mydata['Merch description'].map(merchdesc_merchnum))

In [25]:
mydata['Merchnum'].isnull().sum()

2094

Fill in missing values with mode of Merch zip.

In [29]:
zip_merchnum = {}
for index,zip5 in mydata[mydata['Merch zip'].notnull()]['Merch zip'].items():
    if zip5 not in zip_merchnum:
        zip_merchnum[zip5] = mydata.loc[index, 'Merchnum']

In [30]:
mydata['Merchnum'] = mydata['Merchnum'].fillna(mydata['Merch zip'].map(zip_merchnum))

In [31]:
mydata['Merchnum'].isnull().sum()

1007

Fill in missing values with mode of Merch State.

In [32]:
state_merchnum = {}
for index,state in mydata[mydata['Merch state'].notnull()]['Merch state'].items():
    if state not in state_merchnum:
        state_merchnum[state] = mydata.loc[index, 'Merchnum']

In [33]:
mydata['Merchnum'] = mydata['Merchnum'].fillna(mydata['Merch state'].map(state_merchnum))

In [34]:
mydata['Merchnum'].isnull().sum()

416

Assign unknown to the adjustment transactions.

In [35]:
mydata['Merchnum'] = mydata['Merchnum'].mask(mydata['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'Unk')
mydata['Merchnum'] = mydata['Merchnum'].mask(mydata['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'Unk')

In [36]:
mydata['Merchnum'].isnull().sum()

416

Fill in any remaining values with Unknown.

In [37]:
mydata['Merchnum'].fillna('Unk', inplace=True)

Check to see if there are any missing Merchnum values remaining.

In [38]:
mydata['Merchnum'].isnull().sum()

0

### Merch state

Clean in this order:
- If the record has a zip, use the state for that zip, if known.
- If in range 00600-00799, 00900-00999: state=PR
- Use the mode of the merchnum or merch description

Identify the null values:

In [39]:
null_state = mydata[mydata['Merch state'].isnull()]
print(null_state.index)
print(len(null_state))
null_state.head()

Int64Index([  262,   272,   400,   476,   487,   515,   544,   557,   737,
              739,
            ...
            95996, 96303, 96304, 96313, 96345, 96360, 96563, 96603, 96651,
            96730],
           dtype='int64', length=1020)
1020


,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
262,263,5142257575,2010-01-04,Unk,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,320.00,0
272,273,5142124791,2010-01-04,Unk,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,970.00,0
400,401,5142276099,2010-01-04,Unk,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,82.59,0
476,477,5142267793,2010-01-05,Unk,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,17.59,0
487,488,5142267793,2010-01-05,Unk,RETAIL CREDIT ADJUSTMENT,NaN,NaN,P,19.69,0


#### Records that have a zip.

In [40]:
null_state_has_zip = null_state[null_state['Merch zip'].notnull()]
print(null_state_has_zip.index)
print(len(null_state_has_zip))
null_state_has_zip.head()

Int64Index([ 3258,  3262,  3540,  3642,  4969,  5229,  5636,  5900,  6030,
             6136, 12152, 13558, 15019, 15124, 19330, 23374, 23490, 23620,
            24185, 24226, 24283, 24297, 25271, 25442, 25487, 26816, 26922,
            31712, 36538, 36789, 38034, 39023, 41267, 43466, 44407, 44563,
            46054, 47198, 50127, 51057, 51883, 52186, 56632, 56943, 57296,
            57661, 57794, 57981, 58138, 59094, 59270, 59479, 59582, 59825,
            59832, 59908, 60003, 60035, 60168, 60207, 60243, 60338, 60596,
            60901, 61417, 61430, 62420, 62657, 64882, 65084, 66157, 66182,
            66252, 69422, 70551, 70588, 72400, 72447, 73628, 74805, 77608,
            82782, 83272, 83425, 83454, 86405, 90259],
           dtype='int64')
87


,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
3258,3259,5142153880,2010-01-14,582582822587,DIGITAL TECHNOLOGY CONTRA,NaN,926.0,P,2340.0,0
3262,3263,5142154098,2010-01-14,582582822587,DIGITAL TECHNOLOGY CONTRA,NaN,926.0,P,2387.0,0
3540,3541,5142154098,2010-01-17,582582822587,DIGITAL TECHNOLOGY CONTRA,NaN,926.0,P,2300.0,0
3642,3643,5142153880,2010-01-17,582582822587,DIGITAL TECHNOLOGY CONTRA,NaN,926.0,P,2500.0,0
4969,4970,5142194136,2010-01-24,597597721468,CRISTALIA ACQUISITION COR,NaN,929.0,P,83.0,0


Fill these state values in based on their zip. Start first with the PR zip codes.

In [41]:
for index in null_state_has_zip.index:
    if ((mydata.loc[index,'Merch zip'] >= 600) & (mydata.loc[index,'Merch zip'] <= 799)) | ((mydata.loc[index,'Merch zip'] >= 900)&(mydata.loc[index,'Merch zip'] <= 999)):
        mydata.loc[index,'Merch state'] = 'PR'

In [42]:
mydata.loc[3258,'Merch state']

'PR'

Now the non PR zip codes.

In [43]:
null_state_has_zip_nonPR = null_state_has_zip[(null_state_has_zip['Merch zip'] < 600)|
                   ((null_state_has_zip['Merch zip'] > 799) & (null_state_has_zip['Merch zip'] < 900))|
                   (null_state_has_zip['Merch zip'] > 999)]
print(null_state_has_zip_nonPR.index)
null_state_has_zip_nonPR.head()

Int64Index([ 5900,  6136, 12152, 13558, 15019, 15124, 19330, 23374, 23490,
            26816, 36538, 36789, 38034, 39023, 43466, 44407, 44563, 46054,
            47198, 50127, 51057, 51883, 52186, 57296, 57661, 59832, 60003,
            60035, 60901, 62657, 65084, 66157, 66182, 66252, 69422, 70551,
            70588, 72400, 72447, 73628, 74805, 77608, 82782, 83272, 83425,
            83454, 90259],
           dtype='int64')


,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
5900,5901,5142190418,2010-01-27,Unk,ATOMIC ENERGY AGENCY,NaN,1400.0,P,40.00,0
6136,6137,5142272703,2010-01-27,Unk,SHENAV HADRAHOT,NaN,65132.0,P,199.00,0
12152,12153,5142214551,2010-02-18,62074800,ECOMED VERLAGSGESELLSCHAF,NaN,86899.0,P,499.15,0
13558,13559,5142190147,2010-02-25,2582360,UABCS,NaN,23080.0,P,1748.25,0
15019,15020,5142227402,2010-03-02,Unk,SCANDINAVIAN1175745333760,NaN,60528.0,P,1867.58,0


In [44]:
null_state_has_zip_nonPR['Merch zip'].value_counts()

6.0        5
12108.0    5
9.0        4
60528.0    4
2.0        4
86899.0    3
38117.0    3
23080.0    2
1.0        2
801.0      2
48700.0    2
90805.0    2
31040.0    1
76302.0    1
65132.0    1
50823.0    1
3.0        1
95461.0    1
8.0        1
1400.0     1
41160.0    1
Name: Merch zip, dtype: int64

With the list of zip codes, we will fill the state values in manually now. I verified the zip codes on the USPS website. If USPS said "invalid zip code," we will put the Merch state as the mode of Merch description.

In [45]:
for index in null_state_has_zip_nonPR.index:
    if (mydata.loc[index,'Merch zip'] == 12108.0):
        # Lake Pleasent, NY
        mydata.loc[index,'Merch state'] = 'NY'
    elif (mydata.loc[index,'Merch zip'] == 38117.0):
        # Memphis, TN
        mydata.loc[index,'Merch state'] = 'TN'    
    elif (mydata.loc[index,'Merch zip'] == 90805.0):
        # Long Beach/Lakewood, CA
        mydata.loc[index,'Merch state'] = 'CA'
    elif (mydata.loc[index,'Merch zip'] == 31040.0):
        # Dublin, GA
        mydata.loc[index,'Merch state'] = 'GA'        
    elif (mydata.loc[index,'Merch zip'] == 76302.0):
        # Wichita Falls, TX
        mydata.loc[index,'Merch state'] = 'TX'        
    elif (mydata.loc[index,'Merch zip'] == 95461.0):
        # Middletown/Loch Lomond, CA
        mydata.loc[index,'Merch state'] = 'CA'
    elif (mydata.loc[index,'Merch zip'] == 41160.0):
        # Mazie, KY
        mydata.loc[index,'Merch state'] = 'KY'

#### Transactions that do not have a zip code.

Need to update our list of null states.

In [46]:
null_state = mydata[mydata['Merch state'].isnull()]
print(null_state.index)
print(len(null_state))
null_state.head()

Int64Index([  262,   272,   400,   476,   487,   515,   544,   557,   737,
              739,
            ...
            95996, 96303, 96304, 96313, 96345, 96360, 96563, 96603, 96651,
            96730],
           dtype='int64', length=966)
966


,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
262,263,5142257575,2010-01-04,Unk,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,320.00,0
272,273,5142124791,2010-01-04,Unk,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,970.00,0
400,401,5142276099,2010-01-04,Unk,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,82.59,0
476,477,5142267793,2010-01-05,Unk,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,17.59,0
487,488,5142267793,2010-01-05,Unk,RETAIL CREDIT ADJUSTMENT,NaN,NaN,P,19.69,0


Let's fill in these state values with the mode of the merch description.

In [47]:
merchdesc_state = {}
for index,merchdesc in mydata[mydata['Merch description'].notnull()]['Merch description'].items():
    if merchdesc not in merchdesc_state:
        merchdesc_state[merchdesc] = mydata.loc[index,'Merch state']

To fill even more missing values, let's do the same process above but for merchant number now.

In [48]:
merchnum_state = {}
for index,merchnum in mydata[mydata['Merchnum'].notnull()]['Merchnum'].items():
    if merchnum not in merchnum_state:
        merchnum_state[merchnum] = mydata.loc[index,'Merch state']

Fill in the missing merchant state values by mapping.

In [50]:
mydata['Merch state'] = mydata['Merch state'].fillna(mydata['Merch description'].map(merchdesc_state))
mydata['Merch state'] = mydata['Merch state'].fillna(mydata['Merchnum'].map(merchnum_state))

Check to see if there are any null values left in Merch State.

In [51]:
mydata['Merch state'].isnull().sum()

950

Assign unknown to the adjustment transactions.

In [52]:
mydata['Merch state'] = mydata['Merch state'].mask(mydata['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'Unk')
mydata['Merch state'] = mydata['Merch state'].mask(mydata['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'Unk')
                                                                 

Check to see if there are any null values left in Merch State.

In [53]:
mydata['Merch state'].isnull().sum()

298

Fill in the missing values with Unknown.

In [54]:
mydata['Merch state'].fillna('Unk', inplace=True)

In [55]:
mydata['Merch state'].isnull().sum()

0

### Merch zip

In [56]:
null_zip = mydata[mydata['Merch zip'].isnull()]
print(null_zip.index)
print(len(null_zip))
null_zip.head()

Int64Index([   51,    54,    55,    58,    59,    60,    61,    62,    64,
               65,
            ...
            96590, 96601, 96603, 96627, 96647, 96651, 96668, 96720, 96730,
            96734],
           dtype='int64', length=4300)
4300


,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
51,52,5142204384,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,20.15,0
54,55,5142146340,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,23.90,0
55,56,5142260984,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,19.95,0
58,59,5142204384,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,20.15,0
59,60,5142204384,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,20.15,0


None of the records missing their zip code are also missing their states (we already filled in the states.

In [57]:
null_zip['Merch state'].isnull().sum()

0

In [58]:
null_zip['Merch state'].value_counts().head()

VA     1090
Unk     961
NY      314
MD      247
GA      221
Name: Merch state, dtype: int64

Fill missing values using the mode of merchant number.

In [61]:
merchnum_zip = {}
for index, merchnum in mydata[mydata['Merchnum'].notnull()]['Merchnum'].items():
    if merchnum not in merchnum_zip:
        merchnum_zip[merchnum] = mydata.loc[index, 'Merch zip']

In [62]:
mydata['Merch zip'] = mydata['Merch zip'].fillna(mydata['Merchnum'].map(merchnum_zip))

In [63]:
null_zip = mydata[mydata['Merch zip'].isnull()]
print(null_zip.index)
print(len(null_zip))
null_zip.head()

Int64Index([   51,    54,    55,    58,    59,    60,    61,    62,    64,
               65,
            ...
            96434, 96545, 96563, 96603, 96647, 96651, 96668, 96720, 96730,
            96734],
           dtype='int64', length=2412)
2412


,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
51,52,5142204384,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,20.15,0
54,55,5142146340,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,23.90,0
55,56,5142260984,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,19.95,0
58,59,5142204384,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,20.15,0
59,60,5142204384,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,20.15,0


Fill remaining missing values using the mode of merchant description.

In [64]:
merchdesc_zip = {}
for index, merchdesc in mydata[mydata['Merch description'].notnull()]['Merch description'].items():
    if merchdesc not in merchdesc_zip:
        merchdesc_zip[merchdesc] = mydata.loc[index, 'Merch zip']

In [65]:
mydata['Merch zip'] = mydata['Merch zip'].fillna(mydata['Merch description'].map(merchdesc_zip))

Assign unknown to the adjustment transactions.

In [66]:
mydata['Merch zip'] = mydata['Merch zip'].mask(mydata['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'Unk')
mydata['Merch zip'] = mydata['Merch zip'].mask(mydata['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'Unk')
                                           

Check to see if there are any null values left in Merch zip.

In [67]:
mydata['Merch zip'].isnull().sum()

1720

Fill remaining missing merch zip values with the mode of merch state.

In [68]:
state_zip = {}
for index, state in mydata[mydata['Merch state'].notnull()]['Merch state'].items():
    if state not in state_zip:
        state_zip[state] = mydata.loc[index, 'Merch zip']

In [69]:
mydata['Merch zip'] = mydata['Merch zip'].fillna(mydata['Merch state'].map(state_zip))

In [70]:
mydata['Merch zip'].isnull().sum()

495

Fill all remaining missing merch zip values with an unknown.

In [71]:
mydata.fillna('Unk', inplace=True)

Let's check to see if there are any remaining null values.

In [72]:
mydata['Merch zip'].isnull().sum()

0

### Reviewing cleaned data

In [73]:
mydata.shape

(96397, 10)

In [74]:
mydata['Transtype'].value_counts()

P    96397
Name: Transtype, dtype: int64

In [75]:
mydata['Merch state'].value_counts()

TN    12038
VA     7849
CA     6823
IL     6508
MD     5397
      ...  
NS        5
AB        5
QC        4
MB        3
US        1
Name: Merch state, Length: 61, dtype: int64

In [76]:
(mydata['Merch state'].isnull() | mydata['Merch state']==0).sum()

0

In [77]:
mydata['Merch zip'].value_counts()

38118.0    11868
22202.0     2172
63103.0     1650
Unk         1465
8701.0      1272
           ...  
2910.0         1
1028.0         1
33801.0        1
98844.0        1
92879.0        1
Name: Merch zip, Length: 4567, dtype: int64

In [78]:
(mydata['Merch zip'].isnull() | mydata['Merch zip']==0).sum()

0

In [79]:
mydata['Merchnum'].value_counts()

930090121224     9310
5509006296254    2131
9900020006406    1714
Unk              1108
602608969534     1107
                 ... 
4588189810001       1
17548454638         1
166000298756        1
980963              1
95906687331         1
Name: Merchnum, Length: 13090, dtype: int64

In [80]:
(mydata['Merchnum'].isnull() | mydata['Merchnum']==0).sum()

0

In [81]:
mydata.to_csv('clean_card_transactions.csv')